Based off the resource at https://neo4j.com/docs/graph-data-science-client/current/tutorials/load-data-via-graph-construction/

In [54]:
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience # Load neo4j graph data science library
import os
if os.getcwd().split('\\')[-1] == 'notebooks':
    os.chdir('..')

In [15]:
# Connect to a Neo4j instance (assumes local right now)
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")
NEO4J_AUTH = None
NEO4J_DB = os.environ.get("NEO4J_DB", "neo4j")
if os.environ.get("NEO4J_USER") and os.environ.get("NEO4J_PASSWORD"):
    NEO4J_AUTH = (
        os.environ.get("NEO4J_USER"),
        os.environ.get("NEO4J_PASSWORD"),
    )
else:
    NEO4J_AUTH = ("neo4j", "Bookings")
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH, database=NEO4J_DB)

In [90]:
# Load the data
properties = pd.read_csv("Inputs/property_data.csv")
bookings = pd.read_csv("Inputs/bookings_data.csv")

In [94]:
# Start with a fresh database
gds.run_cypher("MATCH (n) DETACH DELETE n")

# Create Database Constraints
gds.run_cypher("""
CREATE CONSTRAINT unique_properties IF NOT EXISTS
FOR (p:Property) REQUIRE p.id IS UNIQUE
""")

gds.run_cypher("""
CREATE CONSTRAINT unique_countries IF NOT EXISTS
FOR (c:Country) REQUIRE c.name IS UNIQUE
""")

gds.run_cypher("""
CREATE CONSTRAINT unique_cities IF NOT EXISTS
FOR (c:City) REQUIRE c.name IS UNIQUE
""")

gds.run_cypher("""
CREATE CONSTRAINT unique_property_types IF NOT EXISTS
FOR (t:Type) REQUIRE t.type IS UNIQUE
""")

""


In [104]:
property_data_dict = {"properties": properties.to_dict('records')}

In [105]:
property_setup_query = """
UNWIND $properties AS property
MERGE (p:Property {id: property.id})
SET p.capacity = property.capacity
SET p.pets_allowed = property.pets_allowed
MERGE (t:Type {name: property.property_type})
MERGE (ct:City {name: property.city})
MERGE (c:Country {name: property.country})
MERGE (p)-[:LOCATED_IN]->(ct)
MERGE (ct)-[:LOCATED_IN]->(c)
MERGE (p)-[:HAS_TYPE]->(t)
"""

In [106]:
gds.run_cypher(property_setup_query, property_data_dict)

""


In [107]:
properties

,property_type,country,city,capacity,pets_allowed,id
0,condo,ITALY,Naples,6,yes,0
1,condo,ITALY,Naples,6,yes,1
2,apartment,FRANCE,Rouen,6,no,2
3,house,SPAIN,Madrid,8,no,3
4,condo,ITALY,Rome,6,yes,4
...,...,...,...,...,...,...
995,condo,BELGIUM,Ghent,4,no,995
996,condo,FRANCE,Rouen,6,yes,996
997,apartment,FRANCE,Lyon,8,yes,997
998,apartment,ITALY,Rome,9,no,998
